# Трансформеры
В этом домашнем задании мы рассмотим использование трансформеров в библиотеке PyTorch. Рассмотрим задачу языкового моделирования. Попробуем генерировать текст нейронной сетью. 

Ссылка на данные - https://drive.google.com/drive/folders/1x1A4ElliUGBPnHladGMwPxPuGxI8Vnpu?usp=sharing

In [1]:
# хороший тон, импортировать все необходимые библиотеки в одной ячейке ;)

import torch
from torch import nn

import numpy as np
import time

from collections import Counter
from sklearn.model_selection import train_test_split

import math
from torch.utils.data import DataLoader

Что такое языковое моделирование? Это предсказание вероятности следующего токена (слова или буквы) на основе предыдущих токенов. Математически это можно описать так:

$$P(x_i|x_1, x_2 , ... , x_{i-1})$$ 

Последовательность $$ x_1, x_2, ... x_{i-1} $$ называют контекстом.

## Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
$$ P(раму | мама, мыла) * P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) * P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) * P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) * P(комиксы | я, часто ,читаю) $$
Попробуйте объяснить выбор для каждого из примеров.

Ответ : $$ P(раму | мама, мыла) > P(папу | мама, мыла) $$
$$ P(столу | дорога, ложка, к) < P(обеду | дорога, ложка, к) $$
$$ P(Евпатий | меня, зовут) < P(Ваня | меня, зовут) $$
$$ P(журналы | я, часто ,читаю) > P(комиксы | я, часто ,читаю) $$

1) Слова из скороговорки.  
2) Также скороговорка.  
3) Ваня - более распространенное имя.  
4) Сложно. Но в России чаще читают журналы, а не комиксы. Такая культура

Если для некоторых из примеров проставить знаки достаточно просто, то на некоторые сложно сказать, какой овтет верный. Мы принимаем решение для данного задания исходя их опыта использования русского языка. Мы много читали на русском и слушали огромное количество русской речи. Обучение языковых моделей происходит по схожему принципу. 

Мы хотим показать модели столько текстов, сколько можем и надеемся, что она наберется достаточно опыта, чтобы расставлять такие знаки неравества максимально схоже с человеком.

## Задание 1 (0.5 балла)
Мы будем обучать языковую модель для предсказания следущей буквы. Такие языковые модели применяются в распозновании речи, так как предоставляют дополнительную информацию акустической модели при выборе следующего символа. Для начала, откройте файл с данными, посмотрите, какие символы входят в тексты, сколько их. Уберите из текста все символы переноса на новую строку и табуляцию.

In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/small_corp_for_test.txt'
file = open(path, 'r')
data = file.readlines()
file.close()
print(len(data))

700000


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_string = ""
for line in range(len(data)):
  data_string += data[line]
data_unique = Counter(data_string)
print("Количество уникальных символов:", data_unique) # проверим, есть ли что-нибудь, что нам не надо

Количество уникальных символов: Counter({' ': 4380220, 'о': 2779376, 'а': 2366842, 'е': 2028774, 'т': 1985934, 'н': 1582468, 'и': 1501660, 'с': 1401746, 'в': 1130062, 'р': 1055739, 'л': 915552, 'д': 899550, 'м': 878018, 'у': 848660, 'к': 795405, 'п': 757949, '\n': 700000, 'я': 555131, 'ь': 538119, 'ы': 403604, 'г': 402608, 'б': 384499, 'ч': 370993, 'з': 355889, 'й': 289713, 'ж': 249344, 'ш': 187256, 'э': 162631, 'ю': 149791, 'х': 147781, 'ц': 117824, 'ё': 76849, 'щ': 66697, 'ф': 60354, '-': 43618, 'ъ': 5406})


In [5]:
# теперь удалим перенос строки
data = [st.replace("\n", '') for st in data]

data[:10]

['добро',
 'кого',
 'капитан',
 'нет',
 'зачем',
 'что происходит',
 'что такое',
 'рассказ',
 'никому',
 'ну что']

## Задание 2 (0.5 балла)
Для обучения модели требуется сначала подготовить текст в подходящий для нейросети вид. Важно также отметить, что нужно добавить два токена start и end, которые отвечают за начало и конец текста. Используйте [ и ] для этой задачи. Также нам нужен токен pad, чтобы заполнять им текст до требуемой длинны для формирования батча.

Реализуйте метод preprocess класса Preprocessor. Он должен принимать на вход текст и длинну текста, которую мы ожидаем получить на выходе. Текст должен быть переведен в нижний регистр, в конец текста добавляется требуемое число pad токенов, далее текст векторизуется (каждому символу ставится свое число). Вернуть требуется два вектора. Полученный результат без последнего токена (на нем будем обучаться) и полученный результат без первого токена (целевые метки при обучении).

In [6]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчм фяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        # YOUR CODE HERE
        text = text.lower() # переводим в нижний регистр
        while len(text) != window_size:
          text += "_" # добавляем токены pad
        
        encoded = np.array([self.token2ind[token] for token in text])
        return encoded[:-1], encoded[1:]
        

## Задание 3 (0.5 балла)
Так как мы решили, что текст будет начинаться токеном [ и заканчиваться токеном ], данные нужно поправить. Реализуйте эту идею, добавьте данные токены в ваши тексты.

In [7]:
# YOUR CODE HERE
for line in range(len(data)):
  data[line] = "[" + data[line] + "]"

data[:10]

['[добро]',
 '[кого]',
 '[капитан]',
 '[нет]',
 '[зачем]',
 '[что происходит]',
 '[что такое]',
 '[рассказ]',
 '[никому]',
 '[ну что]']

## Задание 4 (0.5 балла)
Так как мы не располагаем большими мощностями, то давайте ограничим максимальную длинну текста. Вы можете менять этот порог и тем самым уменьшать кол-во текстов в вашей выборке и увеличивая тем самым скорость обучения. Начнем же мы с 128. 
Выберите порог и оставьте только те тексты, длина которых не превосходит данный порог.

Далее разбейте тексты на train и test, перемешайте тексты при разбиении, размер тестовой выборки должен быть 15% от общего числа текстов. 

In [8]:
THRESHOLD = 128
data_lim = []

print("Число до обработки:", len(data))
# YOUR CODE HERE
for line in data:
  if len(line) <= THRESHOLD:
    data_lim.append(line)
print("Число после обработки:", len(data_lim))
# create training and validation data
data_train, data_test = train_test_split(data_lim, random_state=666, train_size=0.85)

Число до обработки: 700000
Число после обработки: 683438


## Задание 5 (2 балла)
Напишем датасет. На вход датасету передается набор текстов, объект класса Preprocessor и размер окна, который вы выбрали в прошлом задании.
Реализуйте методы __len__ и __getitem__.

In [9]:
class TextDataset(torch.utils.data.Dataset):
    
    def __init__(self, x, preproc, win_size = THRESHOLD):
        # YOUR CODE HERE
        self.x = [preproc.preprocess(token, win_size) for token in x]
    
    def __len__(self):
        # YOUR CODE HERE
        return len(self.x)
        
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        return self.x[idx]

In [10]:
preproc = Preprocessor()
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

## Задание 6 (2 балла)
Напишем модель. Класс для реализации positional encoding реализован за вас, он нужен, чтобы модель могла после получения эмбедингов понимать, на каком месте какой токен находится.

Заполните пропуски в классе модели. Гипперпараметры модели вам предлагается подобрать самостоятельно. Рекомендуется использовать не более 6 слоев в трансформере. В декореде испоьлзуйте две линейных слоя с функцией активации ReLU между ними.

## Задание 6_1 (0 баллов, но надо ответить!)
При обучении языковой модели на основе трансформеров мы используем маскирование символов (как мы это делаем - уже реализовано). Напишите, почему мы это делаем? Почему это так важно?

Благодаря максированию, элементы будут реагировать только на входные элементы, которые идут в последовательности до них

In [11]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=1000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.emb = nn.Embedding(vocab_size, 512)
        self.pe = PositionalEncoding(512)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(512, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=6)
        self.decoder = nn.TransformerDecoder(nn.TransformerDecoderLayer(512, nhead=8), num_layers = 6)
    
    def forward(self, x, src_mask):
        x = self.pe(self.emb(x)) # emb, then pe
        x = x.transpose(1, 0)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(x) # decoder
        return x.transpose(1, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [18]:
model = LanguageModel(len('_добсркгаупитнезчм яжлйвцфыэь-шхющёъ][ '))

## Задание 7 (2,5 балла)
Финишная прямая. Давайте реализуем класс для обучения модели и ее валидации. Следуйте указаниям в коде и заполните недостающие фрагменты в коде.

In [14]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.model = model
        
        self.train_batch_size = 127
        self.test_batch_size = 127
        
        self.train_dataloader = DataLoader(train_dataset, batch_size=self.train_batch_size, shuffle = True)
        self.test_dataloader = DataLoader(test_dataset, batch_size=self.train_batch_size)
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        
        self.criterion = nn.CrossEntropyLoss(ignore_index=1) # используйте CrossEntrophyLoss, передайте в качетсве параметра 
                                                               # ignore index индекс символа _, чтобы модель не штрафовалась за то
                                                               # что идет после закрывающего токена
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), 1e-4, betas=(0.9, 0.98), eps=1e-9)
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for src, tgt in self.train_dataloader:
            # YOUR CODE HERE
            src = src.to(self.device)
            tgt = tgt.to(self.device)
            predicted = self.model(src, self.model.generate_square_subsequent_mask(self.train_batch_size).to(self.device))
            loss = self.criterion(predicted, tgt)
            # del src
            # del tgt
            # del predicted
            # torch.cuda.synchronize()
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            counted_loss += loss.item()
            # Update weights
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for src, tgt in self.test_dataloader:
            # YOUR CODE HERE
            src = src.to(self.device)
            tgt = tgt.to(self.device)
            predicted = self.model(src, self.model.generate_square_subsequent_mask(self.test_batch_size).to(self.device))
            loss = self.criterion(predicted, tgt)
            counted_loss += loss.item()
            # del src
            # del tgt
            # del predicted
            # torch.cuda.synchronize()
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Что такое ppl? Перплексия. Ее можно интерпретировать как меру "удивленности" модели нужному символу. Чем меньше данная величина, тем лучше, ведь это значит, что модель если и сделала неправильный выбор, то не сильно удивлена своей ошибке.

Проведите несколько экспериментов, посмотрите, при каких гипперпараметрах значение перплексии минимально.

## Задание 8 (0.5 балла)
Запустите обучение на нескольких эпохах. Ориентируйтесь на ваши вычислительные мощности и время работы. Вы всегда можете посчитать, сколько секунд уходит на один батч.

In [19]:
# YOUR CODE HERE
Trainer(model, train_dataset, test_dataset).train(number_of_epochs=3)
# времени не осталось :(

TypeError: ignored

## Задание 9 (1 балл)
Итак, давайте попробуем погенерировать текст нашей сеткой. Закончите функцию по генерации текста. Попробуйте сгенерировать какой-нибудь текст. Помните, что если вы хотите генерировать текст с нуля, то вы должны передать в качестве текста только токен start.
Прекратите генерировать текст, если модель выдала токен end или длинна текста больше 150.

In [ ]:
def generate_text(text):
    x = []
    
    for letter in text:
        x.append(preproc.token2ind[letter])
    x = torch.from_numpy(np.array(x))
    
    pred = ...
    ind = ... 
    
    text += ... 
    
    if ...:
        return text
    else:
        return generate_text(text)

## Задание 10* (Задание - бонус, 5 баллов за реализацию при условии, что сделаны прошлые задания)
Давайте вспомним, что такое transfer learning. Мы хотим использовать уже предобученные эмбединги для нашей сети, чтобы наша сеть обучалась быстрее. Давайте попробуем обучить новую модель на уровне слов, а не символов, но для упрощения задачи используем предобученный слой из библиотеки Natasha, а вернее, ее блок Navec.

[Изучите](https://github.com/natasha/navec) то, как вставить слой в вашу нейронную сеть.

Теперь мы хотим, чтобы на вход модели подавались слова, модифицируйте ваш датасет. Возвращайте теперь номер слова в словаре navec.

In [ ]:
class TextDataset_Navec(torch.utils.data.Dataset):
    
    def __init__(self, x, win_size = 128):
        # YOUR CODE HERE
        self.navec = ...
        ################
    
    def __len__(self):
        # YOUR CODE HERE
        ################
    
    def __getitem__(self, idx):
        # YOUR CODE HERE
        ################

Немного модифицируем модель. Теперь нам не нужны слои с трансформером, так как весь механизм внимания уже заложен в ембедингах. Давайте попробуем просто пройтись линейной головой над эмбедингами. Выберите параметры самостоятельно.

In [ ]:
class LanguageModel(nn.Module):
    def __init__(self):
        super(LanguageModel, self).__init__()
        self.emb_navec = ...
        self.head = ...
    
    def forward(self, x):
        x = ... # emb
        x = ... # head
        return x

Теперь дело за малым! Надо немного модифицировать класс обучения, так как мы не используем маскирование, после чего можно приступить к тесту!

In [ ]:
class Trainer:
    
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = ...
        self.test_dataloader = ...
        self.train_dataloader_size = ...
        self.test_dataloader_size = ...
        
        self.device = 'cuda:0'
        self.criterion = ... 
        
        self.optimizer = ...
        
        self.steps_to_print = 1000
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        
        for batch in self.train_dataloader:
            x, y = batch
            # YOUR CODE HERE
            
            # реализуйте шаги обучения модели
            # сохраняйте значение ошибки в переменную counted_loss
            
            ################
            
            
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

Запустите обучение. 

In [ ]:
# YOUR CODE HERE
###############